#データ概要
国勢調査は、日本に住んでいるすべての人と世帯を対象とする国の最も重要な統計調査で、５年ごとに実施されます。国勢調査から得られる日本の人口や世帯の実態は、国や地方公共団体の行政において利用されることはもとより、民間企業や研究機関でも広く利用され、そのような利用を通じて国民生活に役立てられています。
　国勢調査では、年齢別の人口、家族構成、働いている人や日本に住んでいる外国人などの結果を提供しています。
//
https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200521&tstat=000001011777&cycle=0&tclass1=000001011778&statdisp_id=0003410379&tclass2val=0

下記の概要

e-Stat APIから人口データを取得し、必要な情報を抽出・整形して、BigQueryにアップロードする処理を行います。Google Cloudの認証設定と、APIレスポンスの処理、BigQueryへのデータロードを実行します。

In [39]:
import requests
import pandas as pd
import os
from google.cloud import bigquery
from google.cloud.exceptions import GoogleCloudError
from typing import Dict, List

# === 定数 ===
PROJECT_ID = "just-episode-454309-a0"
DATASET_ID = "20250413_estat_dataset"
TABLE_ID = "20250413_estat_population"
APP_ID = "コード取得必要"  # 自分の App ID に変更
STATS_DATA_ID = "0003410379"
TARGET_AREAS = ["13000", "14000", "26000", "27000", "40000", "41000"]
AREA_DICT = {
    '40000': '福岡県',
    '13000': '東京都',
    '14000': '神奈川県',
    '26000': '京都府',
    '27000': '大阪府',
    '41000': '佐賀県'
}

# === BigQueryクライアント作成 ===
client = bigquery.Client()

# === 認証情報の設定 ===
# 認証情報設定
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kobayashitoshiyuki/Desktop/estat/ファイル取得必要.json"

def fetch_estat_data(app_id: str, stats_data_id: str) -> dict:
    """e-Stat API から統計データを取得"""
    url = "http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData"
    params = {
        "appId": app_id,
        "lang": "J",
        "statsDataId": stats_data_id,
        "metaGetFlg": "Y",
        "cntGetFlg": "N",
        "explanationGetFlg": "Y",
        "annotationGetFlg": "Y",
        "sectionHeaderFlg": "1",
        "replaceSpChars": "0"
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        raise SystemExit(f"API取得に失敗しました: {e}")

def extract_population_data(data: dict) -> pd.DataFrame:
    """APIレスポンスから人口データを抽出・整形"""
    # 性別マスタ
    sex_class = pd.json_normalize(data['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ'][1])
    sex_expanded = pd.DataFrame(sex_class["CLASS"][0])
    sex_expanded["category_id"] = sex_class["@id"][0]
    sex_expanded["category_name"] = sex_class["@name"][0]
    sex_expanded.columns = sex_expanded.columns.str.replace('@', '', 1)

    # 人口データ
    values = pd.json_normalize(data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
    values.columns = values.columns.str.replace('@', '', 1)
    values.rename(columns={'$': '人数'}, inplace=True)

    # 都道府県・性別など整形
    dat = values.query("area in @TARGET_AREAS").copy()
    dat["県名"] = dat["area"].map(AREA_DICT)
    dat["年"] = dat["time"].astype(str).str[:4]
    dat = dat.merge(sex_expanded, left_on="cat01", right_on="code", how="left")
    dat.rename(columns={'name': '性別'}, inplace=True)
    dat["人数"] = pd.to_numeric(dat["人数"], errors="coerce").fillna(0).astype(int)
    dat = dat[["年", "県名", "性別", "人数"]]

    return dat

def create_dataset_if_not_exists(project_id: str, dataset_id: str) -> None:
    """データセットが存在しない場合に作成"""
    dataset_ref = client.dataset(dataset_id)
    try:
        client.get_dataset(dataset_ref)  # Check if dataset exists
    except GoogleCloudError:
        # Create dataset if it doesn't exist
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = "asia-northeast1"  # Set location
        client.create_dataset(dataset)
        print(f"🆕 データセット {dataset_id} を作成しました。")

def load_to_bigquery(df: pd.DataFrame, project_id: str, dataset_id: str, table_id: str) -> None:
    """DataFrame を BigQuery にロード"""
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    schema = [
        bigquery.SchemaField("年", "STRING"),
        bigquery.SchemaField("県名", "STRING"),
        bigquery.SchemaField("性別", "STRING"),
        bigquery.SchemaField("人数", "INTEGER"),
    ]
    
    # データセットが存在しない場合は作成
    create_dataset_if_not_exists(project_id, dataset_id)

    try:
        job_config = bigquery.LoadJobConfig(
            schema=schema,
            write_disposition="WRITE_TRUNCATE",  # テーブルを上書きする設定
            source_format=bigquery.SourceFormat.CSV,  # CSV形式でデータをロード
            skip_leading_rows=1,  # ヘッダー行をスキップ
            autodetect=False,  # スキーマを手動で指定
        )
        
        # CSV 形式でデータをロード
        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()  # ジョブの完了を待機
        print(f"✅ BigQuery にアップロード完了: {table_ref}")
    except GoogleCloudError as e:
        raise SystemExit(f"BigQuery へのアップロードに失敗しました: {e}")

def main():
    print("🔄 データ取得中...")
    data = fetch_estat_data(APP_ID, STATS_DATA_ID)
    
    print("🧹 データ整形中...")
    final_df = extract_population_data(data)
    
    print("☁️ BigQuery にアップロード中...")
    load_to_bigquery(final_df, PROJECT_ID, DATASET_ID, TABLE_ID)

    print("🎉 処理完了！")

if __name__ == "__main__":
    main()


🔄 データ取得中...
🧹 データ整形中...
☁️ BigQuery にアップロード中...
🆕 データセット 20250413_estat_dataset を作成しました。
✅ BigQuery にアップロード完了: just-episode-454309-a0.20250413_estat_dataset.20250413_estat_population
🎉 処理完了！


下記コードの概要

BigQuery から人口データを取得し、年・都道府県別および年・性別別の平均人数グラフを作成、ローカルに保存し、GitHub に自動でプッシュするワークフローを自動化

In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import subprocess
from datetime import datetime
from google.cloud import bigquery
import db_dtypes

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kobayashitoshiyuki/Desktop/estat/ファイル取得必要.json"

# BigQuery クライアント作成
client = bigquery.Client()


# === 定数 ===
PROJECT_ID = "just-episode-454309-a0"
DATASET_ID = "20250413_estat_dataset"
TABLE_ID = "20250413_estat_population"
LOCAL_FOLDER = "/Users/kobayashitoshiyuki/Desktop/estat/20250413"
GITHUB_REPO_PATH = "/Users/kobayashitoshiyuki/Desktop/estat/20250413"
GITHUB_REMOTE_URL = "https://github.com/g238025t/20250413.git"  # 自分のURLに変更！
# === BigQueryクライアント作成 ===
client = bigquery.Client()

# === BigQueryからデータ取得 ===
def load_data_from_bigquery() -> pd.DataFrame:
    query = f"""
        SELECT *
        FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    """
    return client.query(query).to_dataframe()

# === グラフ作成・保存 ===
def create_and_save_graphs(df: pd.DataFrame, folder_path: str):
    os.makedirs(folder_path, exist_ok=True)

    # フォント設定（日本語対応）
    plt.rcParams["font.family"] = "IPAexGothic"

    # 県ごとの色割り当て
    area_list = df["県名"].astype(str).unique()
    colors = list(mcolors.TABLEAU_COLORS.values())[:len(area_list)]
    area_colors = {area: color for area, color in zip(area_list, colors)}

    # 性別ごとの色
    sex_list = df["性別"].astype(str).unique()
    sex_colors = {sex: color for sex, color in zip(sex_list, ["blue", "red", "green"])}

    # グループ化
    area_year_avg = df.groupby(["年", "県名"])["人数"].mean().sort_index(level="年", ascending=True)
    sex_year_avg = df.groupby(["年", "性別"])["人数"].mean().sort_index(level="年", ascending=True)

    # グラフ①（都道府県別）
    fig1_path = os.path.join(folder_path, "area_year_avg.png")
    plt.figure(figsize=(12, 20))
    plt.barh(
        [f"{year} - {area}" for year, area in area_year_avg.index],
        area_year_avg.values,
        color=[area_colors[area] for _, area in area_year_avg.index]
    )
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=area)
               for area, color in area_colors.items()]
    plt.legend(handles=handles, title="都道府県")
    plt.xlabel("人数")
    plt.ylabel("年 - 県名")
    plt.title("平均人数（年・都道府県別）")
    plt.grid(axis='x', linestyle="--", alpha=0.7)
    plt.savefig(fig1_path, bbox_inches="tight")
    plt.close()

    # グラフ②（性別別）
    fig2_path = os.path.join(folder_path, "sex_year_avg.png")
    plt.figure(figsize=(12, 10))
    plt.barh(
        [f"{year} - {sex}" for year, sex in sex_year_avg.index],
        sex_year_avg.values,
        color=[sex_colors[sex] for _, sex in sex_year_avg.index]
    )
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=sex)
               for sex, color in sex_colors.items()]
    plt.legend(handles=handles, title="性別")
    plt.xlabel("人数")
    plt.ylabel("年 - 性別")
    plt.title("平均人数（年・性別別）")
    plt.grid(axis='x', linestyle="--", alpha=0.7)
    plt.savefig(fig2_path, bbox_inches="tight")
    plt.close()

def initialize_git_repo():
    print("🔧 Git リポジトリを初期化中...")
    os.chdir(GITHUB_REPO_PATH)

    # .git ディレクトリがない場合のみ初期化
    if not os.path.isdir(os.path.join(GITHUB_REPO_PATH, ".git")):
        subprocess.run(["git", "init"])
        subprocess.run(["git", "remote", "add", "origin", GITHUB_REMOTE_URL])
    else:
        print("✅ 既に Git リポジトリとして初期化済み")

# === GitHub へアップロード ===
def push_to_github(repo_path: str):
    os.chdir(repo_path)
    commit_message = f"Update graphs {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

    subprocess.run(["git", "config", "--global", "user.name", "取得必要"])  # ← 自分の名前に変更
    subprocess.run(["git", "config", "--global", "user.email", "取得必要"])  # ← 自分のメールに変更

    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])

    # ブランチ名を取得（例: main, master）
    result = subprocess.run(["git", "branch", "--show-current"], stdout=subprocess.PIPE, text=True)
    current_branch = result.stdout.strip()

    # 初回 push（--set-upstream を付ける）
    subprocess.run(["git", "push", "--set-upstream", "origin", current_branch])


# === メイン処理 ===
def main():
    print("📥 BigQuery からデータ読み込み中...")
    df = load_data_from_bigquery()

    print("📊 グラフ作成中...")
    create_and_save_graphs(df, LOCAL_FOLDER)

    print("🔧 Git リポジトリ初期化中...")
    initialize_git_repo()  

    print("🚀 GitHub にプッシュ中...")
    push_to_github(GITHUB_REPO_PATH)

    print("🎉 完了しました！")

if __name__ == "__main__":
    main()


📥 BigQuery からデータ読み込み中...


/Users/kobayashitoshiyuki/opt/anaconda3/envs/venv/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2518: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/Users/kobayashitoshiyuki/opt/anaconda3/envs/venv/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2532: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/Users/kobayashitoshiyuki/opt/anaconda3/envs/venv/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2546: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/Users/kobayashitoshiyuki/opt/anaconda3/

📊 グラフ作成中...
🔧 Git リポジトリ初期化中...
🔧 Git リポジトリを初期化中...
✅ 既に Git リポジトリとして初期化済み
🚀 GitHub にプッシュ中...
On branch main
nothing to commit, working tree clean
branch 'main' set up to track 'origin/main'.
🎉 完了しました！


To github.com:g238025t/20250413.git
 * [new branch]      main -> main
